# Add document id

In [ ]:
import os
import glob
import time

# 00001 -> 1
def recover_to_int(strnumber):
    for z in range(len(strnumber)):
        if strnumber[z]!=0:
            return int(strnumber[z:])
    return 0


def readentfile(targetdir):
    pos_cui={}
    pos_sem={}
    pos_term={}
    
    ent_mode = False
    file = open(targetdir, "r")
    lines = file.readlines()
    
    documents = []
    sentidx_to_docidx = {}
    
    for l in range(len(lines)):
        line = lines[l].strip()
        
        if "#### entities" in line:
            ent_mode=True
        
        if len(line.split("\t"))<3:
            continue
        
        if ent_mode==True:
            key = line.split("\t")[0]
            pos_cui[key]=line.split("\t")[1]
            pos_sem[key]=line.split("\t")[2]
            pos_term[key]=line.split("\t")[3]
        
        else:
            sentidx = int(line.split("\t")[0])
            docidx = int(line.split("\t")[1])
            
            # Used to add document number to the location of the entity
            # Returns sentence number-document number
            sentidx_to_docidx[sentidx] = docidx
            
            # Sentence number, document number...
            # Change to document number, sentence number...
            rest = "\t".join(line.split("\t")[2:])
            documents.append(str(docidx)+"\t"+str(sentidx)+"\t"+rest)
            
            
    file.close()
    
    return pos_cui, pos_sem, pos_term, sentidx_to_docidx, documents
    

def add_doc_to_key(pos_cui, pos_sem, pos_term, sentidx_to_docidx):
    
    entities_fixed = ["#### entities (문장번호/begin/end) ####"]
    
    for key in pos_cui:
#         print("key: ", key)
        sentid = key.split("/")[0]
        begin = key.split("/")[1]
        end = key.split("/")[2]
        
        docidx = str(sentidx_to_docidx[recover_to_int(sentid)])
        
        zeros_d = "".join(['0']*(8-len(str(docidx))))
        
        newkey = (zeros_d+docidx)+"/"+sentid+"/"+begin+"/"+end
        entities_fixed.append(newkey+"\t"+pos_cui[key]+"\t"+pos_sem[key]+"\t"+pos_term[key])
        
    return entities_fixed
    
    

def file_loop_targeted(department="감염내과", mode="train"):
    start_time = time.time()
    
    cui_to_term = {}
    cui_to_sem = {}
    ent_count = {}
    
    files_input = glob.glob("./data/02_entities_task7/"+str(mode)+"/"+str(department)+"/*_data.txt")
    files_input.sort()
    
    for f in range(0, len(files_input)):
        filename = files_input[f].split("/")[-1]
        print(f, "/", len(files_input))

        pos_cui, pos_sem, pos_term, sentidx_to_docidx, documents = readentfile(targetdir=files_input[f])
        #print("pos_cui: ", pos_cui)
        
        entities_fixed = add_doc_to_key(pos_cui, pos_sem, pos_term, sentidx_to_docidx)
        #print("documents: ", documents)
        #print("entities_fixed: ", entities_fixed)
        
        
        documents = documents + entities_fixed

        outdir = "./data/03_entities_task7_fixed/"+str(mode)+"/"+str(department)
        if not os.path.exists(outdir):
            os.makedirs(outdir)

        file = open(outdir+"/"+str(filename), "w")
        file.write("\n".join(documents))
        file.close()
        


##################################################
## Finetuning data
##################################################
mode="train"
file_loop_targeted(department="감염내과", mode=mode)
file_loop_targeted(department="내분비대사내과", mode=mode)
file_loop_targeted(department="류마티스내과", mode=mode)
file_loop_targeted(department="소화기내과", mode=mode)
file_loop_targeted(department="순환기내과", mode=mode)
file_loop_targeted(department="신장내과", mode=mode)
file_loop_targeted(department="알레르기내과", mode=mode)
file_loop_targeted(department="호흡기내과", mode=mode)

mode="test"
file_loop_targeted(department="감염내과", mode=mode)
file_loop_targeted(department="내분비대사내과", mode=mode)
file_loop_targeted(department="류마티스내과", mode=mode)
file_loop_targeted(department="소화기내과", mode=mode)
file_loop_targeted(department="순환기내과", mode=mode)
file_loop_targeted(department="신장내과", mode=mode)
file_loop_targeted(department="알레르기내과", mode=mode)
file_loop_targeted(department="호흡기내과", mode=mode)

